In [1]:
# DOING: Find ranks of tranco 1M domains with their A records (IP address/es) and save it into a file.
import csv
import pandas as pd
year = "2024"
# 1. Read tranco_domain_a_df that has domain name and ranks: openintel_domain_a_record
# 2. Join it with tranco_domains_with_rank
df1_orig = pd.read_csv("../data/tranco-1m/tranco_domains_with_rank_"+year+".csv")
df1 = df1_orig[0:100000] # Choose only 100k based on https://attentioninsight.com/what-is-alexa-rank-and-its-value/

df2 = pd.read_csv("../data/tranco-1m/openintel_domain_a_record_"+year+".csv")
df3 = df1.merge(df2, how='left', on='domain')

df3.to_csv("../data/tranco-1m/openintel_tranco_resolved_with_rank_100k_"+year+".csv", index=False)

In [2]:
# Find the number of matched IP addresses 

# Compare the ip addresses with the protected prefixes using pytricia loop
import pandas as pd
import ipaddress
import pytricia


def build_prefix_trees(prefix_list):
    """Builds two Patricia Tries: one for IPv4 and one for IPv6."""
    pt_v4 = pytricia.PyTricia()
    pt_v6 = pytricia.PyTricia()

    for prefix in prefix_list:
        network = ipaddress.ip_network(prefix)
        if network.version == 4:

            pt_v4[prefix] = True
        else:
            pt_v6[prefix] = True

    return pt_v4, pt_v6

def count_covered_ips(ip_list, pt_v4, pt_v6):
    """Counts how many IPs are covered by the prefixes in the Patricia Tries."""
    count = 0
    for ip in ip_list:
        ip_obj = ipaddress.ip_address(ip)
        if ip_obj.version == 4 and pt_v4.get(ip):
            count += 1
        elif ip_obj.version == 6 and pt_v6.get(ip):
            count += 1

    return count


def get_covered_ips(ip_list, pt_v4, pt_v6):
    """Returns the list of IPs covered by prefixes in the Patricia Trie."""
    covered_ips = []
    for ip in ip_list:
        ip_obj = ipaddress.ip_address(ip)
        if ip_obj.version == 4 and pt_v4.get(ip):
            covered_ips.append(ip)
        elif ip_obj.version == 6 and pt_v6.get(ip):
            covered_ips.append(ip)
#     covered_ips = [ip for ip in ip_list if prefix_tree.get(ip)]
    return covered_ips



# Protected prefixes lists using all five scrubberscovered_count
df = pd.read_csv("../data/customers_prefixes_scrubber_all_"+year+".csv") 
protected_prefixes = df["prefix"].tolist()

# Openintel list of IP addresses
df = pd.read_csv("../data/tranco-1m/openintel_tranco_resolved_with_rank_100k_"+year+".csv") 
df_cleaned = df.dropna()

tranco_ip_addresses = df_cleaned["ip4_address"].unique()

# Build separate prefix trees for IPv4 and IPv6
pt_v4, pt_v6 = build_prefix_trees(protected_prefixes)

# Find the number of IPs covered
covered_count = count_covered_ips(tranco_ip_addresses, pt_v4, pt_v6)

# Find the IPs covered
covered_ips = get_covered_ips(tranco_ip_addresses, pt_v4, pt_v6)


# Save covered ips in a .txt file.
with open("../data/tranco-1m/openintel_tranco_scrubber_covered_ip_100k_"+year+".txt", "w", encoding="utf-8") as f:
    for ip in covered_ips:
        f.write(f"{ip}\n")
print(f"Number of IPs protected by the five scrubber's protected prefixes is: {len(covered_ips)} and IPs saved in openintel_tranco_scrubber_covered_ip_100k_"+year+".txt")


Number of IPs protected by the five scrubber's protected prefixes is: 878 and IPs saved in openintel_tranco_scrubber_covered_ip_100k_2024.txt


In [3]:
# Find ranks of tranco 1M domains with their A records (IP address/es) and save it into a file.
import csv
import pandas as pd

year = "2023"
covered_ips = []
# Parse massdns results
with open("../data/tranco-1m/openintel_tranco_scrubber_covered_ip_100k_"+year+".txt", "r", encoding="utf-8") as f:
    for line in f:
        ip = line.strip("\n")
        covered_ips.append(ip)
        
# Convert it into a dataframe with column name ipv4_address
df1 = pd.DataFrame(covered_ips, columns=['ip4_address'])

df2 = pd.read_csv("../data/tranco-1m/openintel_tranco_resolved_with_rank_100k_"+year+".csv")

# df3 = df2.merge(df1, how='inner', on='ip4_address')
        
    
result = df1.merge(df2, on='ip4_address', how='inner')
result.to_csv("../data/tranco-1m/openintel_ip_domains_ranks_100k_"+year+".csv", index=False)

print("%s number of domains are protected. \n" %len(result))


3513 number of domains are protected. 



In [1]:
import pandas as pd
df = pd.read_csv("../data/tranco-1m/openintel_ip_domains_ranks_100k_2024.csv")


In [2]:
df

,ip4_address,domain,rank
0,170.114.52.2,zoom.us,76
1,141.193.213.20,applovin.com,240
2,141.193.213.20,forter.com,591
3,141.193.213.20,berkeley.edu,636
4,141.193.213.20,conviva.com,752
...,...,...,...
2810,194.76.18.1,zaimer.kz,99809
2811,194.76.18.2,zaimer.kz,99809
2812,194.76.18.3,zaimer.kz,99809
2813,199.60.103.51,pay.nl,99928
